# 매매가

In [2]:
import requests
import pprint
import pandas as pd
import os

#인증키 입력
encoding = 'AlUCFS55oaHO3YecssapG9Hf0CTFp23Roj27VnlP3ne7Yr34sWeieyrzDTEb1vGbzvjcQhig%2FXjmo1EjP2a1GQ%3D%3D'
decoding = 'AlUCFS55oaHO3YecssapG9Hf0CTFp23Roj27VnlP3ne7Yr34sWeieyrzDTEb1vGbzvjcQhig/Xjmo1EjP2a1GQ=='

#url 입력
url = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcRHTrade?'
service_key = "AlUCFS55oaHO3YecssapG9Hf0CTFp23Roj27VnlP3ne7Yr34sWeieyrzDTEb1vGbzvjcQhig/Xjmo1EjP2a1GQ=="
base_date = "202001" 
gu_code = '11215'

payload = "LAWD_CD=" + gu_code + "&" + \
          "DEAL_YMD=" + base_date + "&" + \
          "serviceKey=" + service_key + "&" 

res = requests.get(url + payload)
print(res)

<Response [200]>


In [3]:
import xml.etree.ElementTree as ET

def get_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list
    
items_list = get_items(res)
items = pd.DataFrame(items_list) 
items

,거래금액,거래유형,건축년도,년,대지권면적,법정동,연립다세대,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"26,000",,2015,2020,22.32,중곡동,미래하이츠,1,2,33.36,,190-31,11215,6,,
1,"21,500",,2000,2020,21.16,중곡동,청기와아트빌2차,1,2,45.56,,111-11,11215,1,,
2,"22,000",,2010,2020,23.14,중곡동,동서화운트빌(88-12),1,4,36.45,,88-12,11215,3,,
3,"28,400",,2003,2020,42.8,중곡동,유진쎄인트빌,1,4,74.92,,82-37,11215,5,,
4,"22,000",,1998,2020,20.76,중곡동,동산빌라,1,6,40.77,,93-9,11215,4,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,"24,500",,2015,2020,17.99,군자동,리치팰리스(158-1),1,8,29.62,,158-1,11215,4,,
151,"28,400",,2019,2020,16.15,군자동,대명이튼캐슬,1,10,24.06,,504,11215,2,,
152,"29,800",,2019,2020,16.15,군자동,대명이튼캐슬,1,10,24.06,,504,11215,5,,
153,"19,500",,2008,2020,23.15,군자동,로즈마리빌,1,20,36.36,,169,11215,3,,


In [4]:
code_file = "./data/법정동코드 전체자료.txt"
code = pd.read_csv(code_file, sep='\t')

code.columns = ['code', 'name', 'is_exist']
code = code [code['is_exist'] == '존재']

print(code['code'][0])
print(type(code['code'][0])) ## int64타입

## string으로 변경
code['code'] = code['code'].apply(str) 

1100000000
<class 'numpy.int64'>


In [5]:
code

,code,name,is_exist
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
46323,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
46324,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
46325,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
46326,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


In [6]:
# year = [str("%02d" %(y)) for y in range(2017, 2022)]
# month = [str("%02d" %(m)) for m in range(1, 13)]
# base_date_list = ["%s%s" %(y, m) for y in year for m in month ]
base_date_list = ["202111","202112","202201","202202","202203","202204"]

In [30]:
gu_code = "41630"
# gu_code = code[ (code['name'].str.contains(gu) )]
# gu_code = gu_code['code'].reset_index(drop=True)
# gu_code = str(gu_code[0])[0:5]
# print(gu_code)

In [31]:
import requests
import datetime

def get_data(gu_code, base_date):
    url ="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcRHTrade?"
    service_key = "AlUCFS55oaHO3YecssapG9Hf0CTFp23Roj27VnlP3ne7Yr34sWeieyrzDTEb1vGbzvjcQhig/Xjmo1EjP2a1GQ=="    
    payload = "LAWD_CD=" + gu_code + "&" + \
              "DEAL_YMD=" + base_date + "&" + \
              "serviceKey=" + service_key + "&"

    res = requests.get(url + payload)
    
    return res

In [32]:
items_list = []
for base_date in base_date_list:
    res = get_data(gu_code, base_date)
    items_list += get_items(res)
    
len(items_list)

2655

In [33]:
sale_df = pd.DataFrame(items_list) 
sale_df.head()
sale_df.to_csv(os.path.join("./data/경기양주시_매매.csv"), index=False,encoding="euc-kr") 

In [39]:
sale_df = pd.read_csv('./data/경기양주시_매매.csv', thousands=',', encoding='cp949')
sale_df = sale_df[['년', '월', '거래금액']]
sale_df.mean()

년       2019.155179
월          6.471563
거래금액    9854.677966
dtype: float64

In [40]:
import requests
import datetime

def get_data(gu_code, base_date):
    url ="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcRHRent?"
    service_key = "AlUCFS55oaHO3YecssapG9Hf0CTFp23Roj27VnlP3ne7Yr34sWeieyrzDTEb1vGbzvjcQhig/Xjmo1EjP2a1GQ=="    
    payload = "LAWD_CD=" + gu_code + "&" + \
              "DEAL_YMD=" + base_date + "&" + \
              "serviceKey=" + service_key + "&"

    res = requests.get(url + payload)
    
    return res

In [41]:
items_list = []
for base_date in base_date_list:
    res = get_data(gu_code, base_date)
    items_list += get_items(res)
    
len(items_list)

1992

In [37]:
lone_df = pd.DataFrame(items_list) 
lone_df.head()
lone_df.to_csv(os.path.join("./data/경기양주시_전세.csv"), index=False,encoding="euc-kr") 

In [42]:
lone_df = pd.read_csv('./data/경기양주시_전세.csv', thousands=',', encoding='cp949')
lone_df = lone_df[['년','월','월세금액','보증금액']]
lone_df = lone_df[lone_df['월세금액'] == 0].drop(['월세금액'], axis=1).reset_index(drop=True)
lone_df = lone_df['보증금액'].mean()
lone_df

7084.608170844939

# 매매가 데이터 분기별로 나누기

In [247]:
sale_df = pd.read_csv('./data/강서구_매매.csv', thousands=',', encoding='cp949')

In [248]:
sale_df2 = sale_df[['거래금액','년','월']]
sale_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22575 entries, 0 to 22574
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   거래금액    22575 non-null  int64
 1   년       22575 non-null  int64
 2   월       22575 non-null  int64
dtypes: int64(3)
memory usage: 529.2 KB


In [249]:
#2018
condition = (sale_df2.년 == 2017)
sale_17 = sale_df2[condition]

condition = (sale_17.월 == 1) | (sale_17.월 == 2) | (sale_17.월 == 3) | (sale_17.월 == 4) | (sale_17.월 == 5) | (sale_17.월 == 6)
sale_17_1 = sale_17[condition]
condition = (sale_17.월 == 7) | (sale_17.월 == 8) | (sale_17.월 == 9) | (sale_17.월 == 10) | (sale_17.월 == 11) | (sale_17.월 == 12)
sale_17_2 = sale_17[condition]

#2018
condition = (sale_df2.년 == 2018)
sale_18 = sale_df2[condition]

condition = (sale_18.월 == 1) | (sale_18.월 == 2) | (sale_18.월 == 3) | (sale_18.월 == 4) | (sale_18.월 == 5) | (sale_18.월 == 6)
sale_18_1 = sale_18[condition]
condition = (sale_18.월 == 7) | (sale_18.월 == 8) | (sale_18.월 == 9) | (sale_18.월 == 10) | (sale_18.월 == 11) | (sale_18.월 == 12)
sale_18_2 = sale_18[condition]

#2019
condition = (sale_df2.년 == 2019)
sale_19 = sale_df2[condition]

condition = (sale_19.월 == 1) | (sale_19.월 == 2) | (sale_19.월 == 3) | (sale_19.월 == 4) | (sale_19.월 == 5) | (sale_19.월 == 6)
sale_19_1 = sale_19[condition]
condition = (sale_19.월 == 7) | (sale_19.월 == 8) | (sale_19.월 == 9) | (sale_19.월 == 10) | (sale_19.월 == 11) | (sale_19.월 == 12)
sale_19_2 = sale_19[condition]

#2020
condition = (sale_df2.년 == 2020)
sale_20 = sale_df2[condition]

condition = (sale_20.월 == 1) | (sale_20.월 == 2) | (sale_20.월 == 3) | (sale_20.월 == 4) | (sale_20.월 == 5) | (sale_20.월 == 6)
sale_20_1 = sale_20[condition]
condition = (sale_20.월 == 7) | (sale_20.월 == 8) | (sale_20.월 == 9) | (sale_20.월 == 10) | (sale_20.월 == 11) | (sale_20.월 == 12)
sale_20_2 = sale_20[condition]

#2021
condition = (sale_df2.년 == 2021)
sale_21 = sale_df2[condition]

condition = (sale_21.월 == 1) | (sale_21.월 == 2) | (sale_21.월 == 3) | (sale_21.월 == 4) | (sale_21.월 == 5) | (sale_21.월 == 6)
sale_21_1 = sale_21[condition]
condition = (sale_21.월 == 7) | (sale_21.월 == 8) | (sale_21.월 == 9) | (sale_21.월 == 10) | (sale_21.월 == 11) | (sale_21.월 == 12)
sale_21_2 = sale_21[condition]

In [250]:
grouped_17_1 = sale_17_1.groupby('년').mean()

grouped_17_2 = sale_17_2.groupby('년').mean()
grouped_18_1 = sale_18_1.groupby('년').mean()
grouped_18_2 = sale_18_2.groupby('년').mean()
grouped_19_1 = sale_19_1.groupby('년').mean()
grouped_19_2 = sale_19_2.groupby('년').mean()
grouped_20_1 = sale_20_1.groupby('년').mean()
grouped_20_2 = sale_20_2.groupby('년').mean()
grouped_21_1 = sale_21_1.groupby('년').mean()
grouped_21_2 = sale_21_2.groupby('년').mean()

sale_concat = pd.concat([grouped_17_1, grouped_17_2, grouped_18_1, grouped_18_2, grouped_19_1, grouped_19_2, grouped_20_1, grouped_20_2, grouped_21_1, grouped_21_2])
sale_concat['분기'] = ['17_1','17_2','18_1','18_2','19_1','19_2','20_1','20_2','21_1','21_2']
sale_concat = sale_concat.reset_index().drop(['년','월'], axis=1)
sale_concat.columns = ['매매가', '분기']
sale_concat.set_index('분기', inplace=True)
sale_concat

,매매가
분기,
17_1,18114.309828
17_2,19034.193608
18_1,19228.439225
18_2,20188.560780
19_1,19408.177547
19_2,20448.231707
20_1,20577.007531
20_2,22247.025189
21_1,23428.909533


# 전세가 데이터 분기별

In [228]:
lone_df = pd.read_csv('./data/강서구_전세.csv', thousands=',', encoding='cp949')
lone_df = lone_df[['년','월','월세금액','보증금액']]
lone_df = lone_df[lone_df['월세금액'] == 0].drop(['월세금액'], axis=1).reset_index(drop=True)
lone_df

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11768\1251632554.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  lone_df = pd.read_csv('./data/강서구_전세.csv', thousands=',', encoding='cp949')


,년,월,보증금액
0,2017,1,16500
1,2017,1,18000
2,2017,1,22000
3,2017,1,25500
4,2017,1,18000
...,...,...,...
33934,2021,12,15000
33935,2021,12,10000
33936,2021,12,15000
33937,2021,12,15000


In [229]:
#2018
condition = (lone_df.년 == 2017)
sale_17 = lone_df[condition]

condition = (sale_17.월 == 1) | (sale_17.월 == 2) | (sale_17.월 == 3) | (sale_17.월 == 4) | (sale_17.월 == 5) | (sale_17.월 == 6)
sale_17_1 = sale_17[condition]
condition = (sale_17.월 == 7) | (sale_17.월 == 8) | (sale_17.월 == 9) | (sale_17.월 == 10) | (sale_17.월 == 11) | (sale_17.월 == 12)
sale_17_2 = sale_17[condition]

#2018
condition = (lone_df.년 == 2018)
sale_18 = lone_df[condition]

condition = (sale_18.월 == 1) | (sale_18.월 == 2) | (sale_18.월 == 3) | (sale_18.월 == 4) | (sale_18.월 == 5) | (sale_18.월 == 6)
sale_18_1 = sale_18[condition]
condition = (sale_18.월 == 7) | (sale_18.월 == 8) | (sale_18.월 == 9) | (sale_18.월 == 10) | (sale_18.월 == 11) | (sale_18.월 == 12)
sale_18_2 = sale_18[condition]

#2019
condition = (lone_df.년 == 2019)
sale_19 = lone_df[condition]

condition = (sale_19.월 == 1) | (sale_19.월 == 2) | (sale_19.월 == 3) | (sale_19.월 == 4) | (sale_19.월 == 5) | (sale_19.월 == 6)
sale_19_1 = sale_19[condition]
condition = (sale_19.월 == 7) | (sale_19.월 == 8) | (sale_19.월 == 9) | (sale_19.월 == 10) | (sale_19.월 == 11) | (sale_19.월 == 12)
sale_19_2 = sale_19[condition]

#2020
condition = (lone_df.년 == 2020)
sale_20 = lone_df[condition]

condition = (sale_20.월 == 1) | (sale_20.월 == 2) | (sale_20.월 == 3) | (sale_20.월 == 4) | (sale_20.월 == 5) | (sale_20.월 == 6)
sale_20_1 = sale_20[condition]
condition = (sale_20.월 == 7) | (sale_20.월 == 8) | (sale_20.월 == 9) | (sale_20.월 == 10) | (sale_20.월 == 11) | (sale_20.월 == 12)
sale_20_2 = sale_20[condition]

#2021
condition = (lone_df.년 == 2021)
sale_21 = lone_df[condition]

condition = (sale_21.월 == 1) | (sale_21.월 == 2) | (sale_21.월 == 3) | (sale_21.월 == 4) | (sale_21.월 == 5) | (sale_21.월 == 6)
sale_21_1 = sale_21[condition]
condition = (sale_21.월 == 7) | (sale_21.월 == 8) | (sale_21.월 == 9) | (sale_21.월 == 10) | (sale_21.월 == 11) | (sale_21.월 == 12)
sale_21_2 = sale_21[condition]

In [230]:
sale_17_1.groupby('년').mean()

,월,보증금액
년,,
2017,3.487308,15452.006658


In [245]:
grouped_17_1 = sale_17_1.groupby('년').mean()
grouped_17_2 = sale_17_2.groupby('년').mean()
grouped_18_1 = sale_18_1.groupby('년').mean()
grouped_18_2 = sale_18_2.groupby('년').mean()
grouped_19_1 = sale_19_1.groupby('년').mean()
grouped_19_2 = sale_19_2.groupby('년').mean()
grouped_20_1 = sale_20_1.groupby('년').mean()
grouped_20_2 = sale_20_2.groupby('년').mean()
grouped_21_1 = sale_21_1.groupby('년').mean()
grouped_21_2 = sale_21_2.groupby('년').mean()

lone_concat = pd.concat([grouped_17_1, grouped_17_2, grouped_18_1, grouped_18_2, grouped_19_1, grouped_19_2, grouped_20_1, grouped_20_2, grouped_21_1, grouped_21_2])
lone_concat['분기'] = ['17_1','17_2','18_1','18_2','19_1','19_2','20_1','20_2','21_1','21_2']
lone_concat = lone_concat.reset_index().drop(['년','월'], axis=1)
lone_concat.columns = ['전세가', '분기']
lone_concat.set_index('분기', inplace=True)
lone_concat

,전세가
분기,
17_1,15452.006658
17_2,16611.562926
18_1,16940.670189
18_2,17681.600977
19_1,17806.894778
19_2,18292.611966
20_1,19211.171386
20_2,20545.926764
21_1,21166.827891


In [255]:
data = pd.concat([sale_concat, lone_concat], axis=1)
data['전세가율'] = data['전세가'] / data['매매가']
data['변동률'] =  data['매매가'].loc['19_1'] / data['매매가'].loc['18_2']
data

,매매가,전세가,전세가율,변동률
분기,,,,
17_1,18114.309828,15452.006658,0.853028,0.961345
17_2,19034.193608,16611.562926,0.872722,0.961345
18_1,19228.439225,16940.670189,0.881022,0.961345
18_2,20188.560780,17681.600977,0.875823,0.961345
19_1,19408.177547,17806.894778,0.917494,0.961345
19_2,20448.231707,18292.611966,0.894582,0.961345
20_1,20577.007531,19211.171386,0.933623,0.961345
20_2,22247.025189,20545.926764,0.923536,0.961345
21_1,23428.909533,21166.827891,0.903449,0.961345
